# Estrattore delle comunicazioni
Il sequente flusso rappresenta il primo estrattore che consente di collezionare i dati dal sito dell'[ANAC](https://dati.anticorruzione.it/). 
Le sorgente di dati sono in formato aperto, in materia di anticorruzione, trasparenza e contratti pubblici acquisiti dall’ANAC nell’ambito delle proprie attività istituzionali.

I dati sono organizzati in dataset visualizzabili in anteprima in forma tabellare e sono scarcabili in formati aperti e standardizzati JSON e CSV e XML.

Gli anni di riferimento sono 2015,2016, 2017 raggiungibile ai seguenti url:

- http://dati.anticorruzione.it/data/l190-2015.json
- http://dati.anticorruzione.it/data/l190-2016.json
- http://dati.anticorruzione.it/data/l190-2017.json

In [1]:
#Import
import pandas as pd
import numpy as np
import os
import csv
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import matplotlib

In [2]:
def get_anac_data(path_to_file, url, force_download=False):
    """
    Carica i dati json dal sito dell'anac, solo se non esistono in locale
    
    Parameters
    ---------
    path_to_file: string
        Path dove trovare il file json o dove memorizzarlo
    url: string
        Url del sito dell'anac
    force_download: boolean
        Se settato a true forza il download
    
    """
    if force_download or not os.path.exists(path_to_file):
        urlretrieve(url, path_to_file)
    df = pd.read_json(path_to_file)
    return df

## Extract

In [3]:
# EXTRACT
# =============================
# Carica i dati 2015, 2016, 2017
df_15 = get_anac_data('../data/l190-2015.json', 'http://dati.anticorruzione.it/data/l190-2015.json')
df_16 = get_anac_data('../data/l190-2016.json', 'http://dati.anticorruzione.it/data/l190-2016.json')
df_17 = get_anac_data('../data/l190-2017.json', 'http://dati.anticorruzione.it/data/l190-2017.json')
# Unisci in un unico dataframe
comunicazioni = pd.concat([df_15,df_16,df_17], ignore_index=True)
comunicazioni.head()

,codiceFiscale,dataUltimoTentativoAccessoUrl,esitoUltimoTentativoAccessoUrl,identificativoPEC,ragioneSociale,url
0,80003710789,2015-04-22T18:42:06.137+0000,fallito,opec275.20140526151617.31733.02.1.48@pec.aruba.it,AMMINISTRAZIONE PROVINCIALE DI COSENZA,http://servizi.provincia.cs.it/legge190/2013/i...
1,80015990544,2015-04-22T13:25:54.003+0000,fallito,opec275.20150112133830.12508.04.1.6@pec.actali...,DIREZIONE DIDATTICA DI CORCIANO,http://www.circolodidatticocorciano.gov.it/?pa...
2,84002280547,2015-04-02T15:21:41.117+0000,successo,opec275.20150110123012.28213.03.1.5@pec.actali...,"ISTITUTO COMPRENSIVO ""G.G. PONTANO"" CERRETO DI...",http://omnicomprensivocerretodispoleto.it/avcp...
3,8876071005,2015-03-31T12:32:19.453+0000,successo,opec275.20150112155450.27942.04.1.16@pec.aruba.it,CONSORZIO INVIOLATELLA SALARIA,http://www.consorzioinviolatellasalaria.it/AVC...
4,91011920179,2015-04-22T16:10:28.597+0000,fallito,opec275.20150109120653.25814.06.2.6@pec.actali...,"ISTITUTO ISTRUZIONE SUPERIORE "" C. MARZOLI """,http://www.istitutomarzoli.gov.it/datil190anno...


Il Data frame contiene i seguenti attributi:

- `codice fiscale` rappresentante la partita IVA o codice fiscale della pubblica amministrazione
- `dataUltimoTentativoAccessoUrl`:attributo avente la data dell'ultimo tentativo di comunicazione 
- `esitoUltimoTentativoAccessoUrl`: attributo caratterizzante l'esito della comunicaizone (fallito, successo)
- `identificativoPEC`: attributo rappresentante l'identificativo della pec
- `ragioneSociale`: avente la denominazione dell'ente pubblico
- `url`:  URL comunicato dalla stazione appaltante e punta alla risorsa resa disponibile dalla stessa stazione appaltante presso il proprio sito istituzionale, contenente la specifica degli appalti

Il dataframe `comunicazioni` rappresenta la base di dati, contenente il set di url da cui scaricare i dati degli appalti pubblici. Per tal motivo è indispensabile eseguire un controllo della sintassi dell'url per eliminare eventuali mancanze di sintassi.

### Correzione  della sintassi dell'url
Il seguente blocco di codice è suddiviso in due parti principali, nella prima si va a verificare l'eventuale presenza di spazi e caratteri speciali con conseguente eliminazione, analogamente nella seconda parte, si va a verificare la corretta forma dell'url. 

Riguardo quest'utlima fase si esegue una **assunzione è che ogni url deve cominciare con `http://` o `https://`** pertanto in caso di esistenza di url senza il protocollo, quest'ultimo viene aggiunto.

In [4]:
index_url_wrong=comunicazioni[
           (comunicazioni['url'].str.contains(" ", na=np.nan)==True) |
           (comunicazioni['url'].str.contains("\n", na=np.nan)==True) |
           (comunicazioni['url'].str.contains("\t", na=np.nan)==True)].index

print ('Numero di url con caratteri speciali all\'interno: **',
       index_url_wrong.shape[0],'**')
if index_url_wrong.shape[0] > 0:
    comunicazioni['url']=comunicazioni['url'].str.replace(" ", "").str.strip(' ').str.replace("\t", "").str.replace("\n", "")
    print ('Cancellazione caratteri speciali,  applicata')

Numero di url con caratteri speciali all'interno: ** 3338 **
Cancellazione caratteri speciali,  applicata


In [5]:
# Trova tutti quelli elementi che non soddisfano la condizione http|HTTP(s|S)?\:\/\/* e inserisci protocollo
comunicazioni['url_wrong']=comunicazioni['url'].str.match('http|HTTP(s|S)?\:\/\/*')==False 
comunicazioni['url']=comunicazioni.apply(lambda x: "http://"+x.url if x.url_wrong else x.url, axis=1)
 #Cancella colonna url_wrong
comunicazioni = comunicazioni.drop('url_wrong', 1)

### Splitta comunicazioni avvenute con successo da comunicazioni fallite

In [9]:
# Comunicazioni avvenute con successo: com_suc
# Comunicazioni fallite: com_fal
com_suc=comunicazioni.query('esitoUltimoTentativoAccessoUrl==\'successo\'')
com_fal=comunicazioni.query('esitoUltimoTentativoAccessoUrl==\'fallito\'')

(34546, 6)

### Elimina duplicati da comunicazioni avvenute con successo

In [10]:
com_suc=com_suc.drop_duplicates()

## Loading
Scrittura su disco del file CSV

In [11]:
# Scrivi dataframe
com_suc.to_csv('../data/com_suc.csv', sep=';', index=False, quoting=csv.QUOTE_ALL)
com_fal.to_csv('../data/com_fal.csv', sep=';', index=False, quoting=csv.QUOTE_ALL)